# Clustering Lab

 
Based of the amazing work you did in the Movie Industry you've been recruited to the NBA! You are working as the VP of Analytics that helps support a head scout, Mr. Rooney, for the worst team in the NBA probably the Wizards. Mr. Rooney just heard about Data Science and thinks it can solve all the team's problems!!! He wants you to figure out a way to find players that are high performing but maybe not highly paid that you can steal to get the team to the playoffs! 

In this document you will work through a similar process that we did in class with the NBA data files will be in the canvas assignment, merging them together.

Details: 

- Determine a way to use clustering to estimate based on performance if 
players are under or over paid, generally. 

- Then select players you believe would be best for your team and explain why. Do so in three categories: 
    * Examples that are not good choices (3 or 4) 
    * Several options that are good choices (3 or 4)
    * Several options that could work, assuming you can't get the players in the good category (3 or 4)

- You will decide the cutoffs for each category, so you should be able to explain why you chose them.

- Provide a well commented and clean report of your findings in a separate notebook that can be presented to Mr. Rooney, keeping in mind he doesn't understand...anything. Include a rationale for variables you included in the model, details on your approach and a overview of the results with supporting visualizations. 


Hints:

- Salary is the variable you are trying to understand 
- When interpreting you might want to use graphs that include variables that are the most correlated with Salary
- You'll need to scale the variables before performing the clustering
- Be specific about why you selected the players that you did, more detail is better
- Use good coding practices, comment heavily, indent, don't use for loops unless totally necessary and create modular sections that align with some outcome. If necessary create more than one script,list/load libraries at the top and don't include libraries that aren't used. 
- Be careful for non-traditional characters in the players names, certain graphs won't work when these characters are included.


In [150]:
# Import libraries

import pandas as pd
import numpy as np
from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

In [151]:
# help(pd.read_csv)

In [152]:
# Read in data

nba1 = pd.read_csv('nba_2025.txt', sep = ',') # values appear to be separated by commas

nba2 = pd.read_csv('2025_salaries.csv', header = 1) # there is a header row, so need to make header = 1 or it will count that as a row of data

In [153]:
# Get rid of duplicate rows

nba1 = nba1.drop_duplicates(subset = 'Player', keep = 'first') # drops duplicated rows, and selects by the player column, keeping the first row (which will keep the grouped statistics row as 2TM will go before the team names)
nba2 = nba2.drop_duplicates(subset = 'Player', keep = 'first')

In [154]:
# Merge datasets and drop missing values

nba = pd.merge(nba1, nba2, on = 'Player', how = 'inner') # inner merge to make sure missing data isn't obstructing the model and we have the salaries and stats for all players we're using/analyzing

In [155]:
#Drop variables that will not be needed or are duplicates

good_cols = ['Player', 'Pos', 'G', 'eFG%', 'FT%', 'PTS', 'AST', '2025-26']

df = nba[good_cols]
df = df.dropna() # some rows still missing, but only a few, so we can just get rid of them
df

,Player,Pos,G,eFG%,FT%,PTS,AST,2025-26
0,Shai Gilgeous-Alexander,PG,49.0,0.599,0.892,1558.0,314.0,"$38,333,050"
1,Tyrese Maxey,PG,52.0,0.547,0.889,1503.0,351.0,"$37,958,760"
2,Donovan Mitchell,SG,51.0,0.574,0.853,1478.0,302.0,"$46,394,100"
3,Jaylen Brown,SF,49.0,0.529,0.775,1435.0,229.0,"$53,142,264"
4,Jalen Brunson,PG,50.0,0.541,0.847,1351.0,305.0,"$34,944,001"
...,...,...,...,...,...,...,...,...
408,Brandon Clarke,PF,2.0,0.333,0.500,8.0,1.0,"$12,500,000"
409,Dillon Jones,SF,4.0,0.300,1.000,7.0,3.0,"$2,753,280"
410,Isaac Jones,PF,4.0,0.500,0.500,5.0,1.0,"$1,955,377"
411,Garrett Temple,SG,13.0,0.214,0.500,4.0,4.0,"$2,296,274"


In [156]:
# Slightly alter some columns for use

df['PPG'] = df['PTS'] / df['G'] # make these standardized to the amount of games to make comparison fair since obviously more playtime leads to more points and assists (usually)
df['APG'] = df['AST'] / df['G'] 

df = df.drop(columns = ['PTS', 'AST', 'G']) # don't need these anymore, so get rid of them

df = df.rename(columns = {'2025-26': 'Salary'}) # purely for convenience because typing out 2025-26 is more annoying than just salary

df

,Player,Pos,eFG%,FT%,Salary,PPG,APG
0,Shai Gilgeous-Alexander,PG,0.599,0.892,"$38,333,050",31.795918,6.408163
1,Tyrese Maxey,PG,0.547,0.889,"$37,958,760",28.903846,6.750000
2,Donovan Mitchell,SG,0.574,0.853,"$46,394,100",28.980392,5.921569
3,Jaylen Brown,SF,0.529,0.775,"$53,142,264",29.285714,4.673469
4,Jalen Brunson,PG,0.541,0.847,"$34,944,001",27.020000,6.100000
...,...,...,...,...,...,...,...
408,Brandon Clarke,PF,0.333,0.500,"$12,500,000",4.000000,0.500000
409,Dillon Jones,SF,0.300,1.000,"$2,753,280",1.750000,0.750000
410,Isaac Jones,PF,0.500,0.500,"$1,955,377",1.250000,0.250000
411,Garrett Temple,SG,0.214,0.500,"$2,296,274",0.307692,0.307692


In [157]:
# Scaling

num_cols = ['eFG%', 'FT%', 'PPG', 'APG']

scaler = StandardScaler()
X = scaler.fit_transform(df[num_cols])

In [158]:
# help(KMeans)

In [159]:
#Run the clustering algo with your best guess for K

kmeans = KMeans(n_clusters = 4, random_state = 42, verbose = 1)
kmeans.fit(X)

Initialization complete
Iteration 0, inertia 1046.8259992445376.
Iteration 1, inertia 818.8674211269569.
Iteration 2, inertia 780.9508938318979.
Iteration 3, inertia 765.9950845910673.
Iteration 4, inertia 757.7743763226273.
Iteration 5, inertia 752.6283132362657.
Iteration 6, inertia 750.3460349561974.
Iteration 7, inertia 748.3579794502224.
Iteration 8, inertia 747.7345645007537.
Converged at iteration 8: strict convergence.


,n_clusters,4
,init,'k-means++'
,n_init,'auto'
,max_iter,300
,tol,0.0001
,verbose,1
,random_state,42
,copy_x,True
,algorithm,'lloyd'


In [160]:
#View the results
print(kmeans.labels_)

df['cluster'] = kmeans.labels_

df

[2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 2 3 2
 2 2 2 2 2 2 2 2 2 2 3 2 2 3 2 2 3 3 3 3 2 3 2 3 2 3 2 3 3 2 2 2 2 2 2 3 3
 3 3 2 2 2 2 2 3 2 3 3 2 3 3 3 3 3 2 2 3 3 3 2 3 3 1 3 3 3 2 3 3 1 2 2 3 3
 3 3 3 2 2 1 3 2 3 2 3 2 3 3 3 3 3 1 3 2 2 3 3 3 2 3 2 3 3 1 2 3 1 3 3 2 3
 3 3 3 2 3 2 3 2 3 3 3 3 2 3 3 1 3 3 1 3 3 3 2 0 3 2 2 3 3 1 2 3 3 3 2 3 3
 2 3 0 2 3 3 3 2 2 1 2 3 3 3 3 0 3 3 1 0 3 3 1 3 0 1 0 3 3 0 0 3 3 0 0 1 3
 0 1 0 3 1 2 3 0 0 1 3 3 0 3 3 3 1 1 3 0 3 1 1 1 0 0 1 3 1 1 3 1 1 3 1 3 0
 3 2 1 1 3 1 1 1 3 3 1 3 0 3 1 3 3 3 0 1 3 1 3 1 0 1 0 3 2 3 0 3 1 0 1 3 3
 1 0 1 1 1 3 1 0 1 1 0 0 1 0 0 3 3 0 1 3 1 0 3 0 0 3 3 1 0 1 0 1 3 2 1 0 1
 1 0 0 1 0 1 0 3 3 3 3 0 1 3 1 1 1 3 3 0 0 1 3 0 3 0 0 1 0 1 0 0 3 2 1 0 0
 1 0 0 3 1 1 0 0 3 0 0 1 2 0 3 1 1 0 0 1 1 0 3 0 1 1 0 1 0 0 0 1 1 0 0 1 0
 0]


,Player,Pos,eFG%,FT%,Salary,PPG,APG,cluster
0,Shai Gilgeous-Alexander,PG,0.599,0.892,"$38,333,050",31.795918,6.408163,2
1,Tyrese Maxey,PG,0.547,0.889,"$37,958,760",28.903846,6.750000,2
2,Donovan Mitchell,SG,0.574,0.853,"$46,394,100",28.980392,5.921569,2
3,Jaylen Brown,SF,0.529,0.775,"$53,142,264",29.285714,4.673469,2
4,Jalen Brunson,PG,0.541,0.847,"$34,944,001",27.020000,6.100000,2
...,...,...,...,...,...,...,...,...
408,Brandon Clarke,PF,0.333,0.500,"$12,500,000",4.000000,0.500000,0
409,Dillon Jones,SF,0.300,1.000,"$2,753,280",1.750000,0.750000,0
410,Isaac Jones,PF,0.500,0.500,"$1,955,377",1.250000,0.250000,1
411,Garrett Temple,SG,0.214,0.500,"$2,296,274",0.307692,0.307692,0


In [ ]:
#Create a visualization of the results with 2 or 3 variables that you think will best
#differentiate the clusters

In [ ]:
#Evaluate the quality of the clustering using total variance explained and silhouette scores

In [ ]:
#Determine the ideal number of clusters using the elbow method and the silhouette coefficient

In [ ]:
#Visualize the results of the elbow method

In [ ]:
#Use the recommended number of cluster (assuming it's different) to retrain your model and visualize the results

In [ ]:
#Once again evaluate the quality of the clustering using total variance explained and silhouette scores

In [ ]:
#Use the model to select players for Mr. Rooney to consider

In [ ]:
#Write up the results in a separate notebook with supporting visualizations and 
# an overview of how and why you made the choices you did. This should be at least 
# 500 words and should be written for a non-technical audience.